In [222]:
%%time

import sys
sys.path.insert(1, r'C:\GitHub\uon-dsda-22-y2-group-project-team-1')


import requests

from bs4 import BeautifulSoup as bs

from service.web_scraping import get_your_move_branches

branches = get_your_move_branches()
branches

CPU times: total: 78.1 ms
Wall time: 240 ms


['Abbey Wood',
 'Abington',
 'Andover',
 'Ashford',
 'Baildon',
 'Barnsley',
 'Basingstoke',
 'Beccles',
 'Bedford',
 'Bedworth',
 'Beeston',
 'Belper',
 'Bexleyheath',
 'Blaby',
 'Blackburn',
 'Blackheath',
 'Bognor Regis',
 'Bothwell',
 'Brighton and Hove',
 'Broadstairs',
 'Bromley',
 'Bromsgrove',
 'Burton on Trent',
 'Bury',
 'Camborne',
 'Canterbury',
 'Carlisle',
 'Castle Bromwich',
 'Chadwell Heath',
 'Chapeltown',
 'Chatham',
 'Chester-le-Street',
 'Coalville',
 'Coatbridge',
 'Consett',
 'Cornwall New Homes',
 'Crawcrook',
 'Crewe',
 'Dalgety Bay',
 'Dalkeith',
 'Darlington',
 'Dartford',
 'Darwen',
 'Deal',
 'Denton',
 'Doncaster',
 'Dudley',
 'Dumfries',
 'Dundee',
 'Dunfermline',
 'Dunstable',
 'Durham',
 'Duston',
 'Eastbourne',
 'Edinburgh',
 'Egham',
 'Elgin',
 'Eltham',
 'Exmouth',
 'Falkirk',
 'Faversham',
 'Forest Gate',
 'Gillingham',
 'Glenrothes',
 'Goole',
 'Gorleston',
 'Gravesend',
 'Hamilton',
 'Hastings',
 'Hemel Hempstead',
 'Hinckley',
 'Hitchin',
 'Hoo',
 

In [223]:
def get_rightmove_sold_address(property_data: str) -> str:
    """
    """
    match = re.compile('.+,propertyType').search(property_data)
    _match = match.group()
    _item = ""
    for item in _match.split(",")[:-1]:
        _item += item + ","
    address = _item[:-1]
    return address


def get_rightmove_sold_property_type(property_data: str) -> str:
    """
    """
    match = re.compile('propertyType:.+,bedrooms').search(property_data)
    _match = match.group()
    _pattern = re.compile(':.+,')
    match = _pattern.search(_match)
    property_type = match.group()[1:-1]
    return property_type



def get_rightmove_sold_bedrooms(property_data: str) -> str:
    """
    """
    match = re.compile('bedrooms:.+,images').search(property_data)
    _match = match.group()
    _pattern = re.compile(':.+,')
    match = _pattern.search(_match)
    bedrooms = match.group()[1:-1]
    return bedrooms


def get_rightmove_sold_price_and_date(property_data: str) -> str:
    """
    """
    match = re.compile('transactions:\[.+,location:').search(property_data)
    _match = match.group()
    match = re.compile('{.+}').search(_match)
    sold_info = match.group().split('},{')
    sale_info = []
    for _property in sold_info:
        price_match = re.compile('£.+,dateSold').search(_property)
        price = price_match.group().split(",d")[0]
        date_match = re.compile('dateSold:.+,tenure').search(_property)
        _date = date_match.group().split(":")[1]
        date = _date.split(",")[0]
        sale_info += [(price, date)]
        
    return sale_info



In [230]:
import re

import pandas as pd

def get_rightmove_sold_properties_by_location(branch: str) -> pd.DataFrame:
    """
    """
    URL = f"https://www.rightmove.co.uk/house-prices/{branch}.html?country=england&searchLocation={branch}"
    page = requests.get(URL)
    soup = bs(page.content, "html.parser")
    
    results = soup.findAll('script', string=re.compile("window.__PRELOADED_STATE__"))
    sold_data = False

    for result in results:
        _results = result.text.split(r'{"address":')
        some_results = _results[1:-1] + [_results[-1].split("detailUrl")[0]]
        for _property in range(len(some_results)):
            individual_property = some_results[_property].replace('"', '')
            address = get_rightmove_sold_address(individual_property)
            property_type = get_rightmove_sold_property_type(individual_property)
            bedrooms = get_rightmove_sold_bedrooms(individual_property)
            sale_info = get_rightmove_sold_price_and_date(individual_property)

            for sale in sale_info:
                if sold_data is False:
                    sold_data = pd.DataFrame(
                        {
                            "location": branch,
                            "address": address,
                            "property type": property_type,
                            "bedrooms": bedrooms,
                            "sale price": sale[0],
                            "sale date": sale[1],
                        }, 
                        index=[0]
                    )
                else:
                    new_sold_data = pd.DataFrame(
                        {
                            "location": branch,
                            "address": address,
                            "property type": property_type,
                            "bedrooms": bedrooms,
                            "sale price": sale[0],
                            "sale date": sale[1],
                        },
                        index=[0]
                    )
                    sold_data = pd.concat(
                        [sold_data, new_sold_data]
                    )
    if sold_data is False:
        return pd.DataFrame(
            {
                "location": [0],
                "address": [0],
                "property type": [0],
                "bedrooms": [0],
                "sale price": [0],
                "sale date": [0],
            },
            index=[0]
        )
    sold_data.reset_index(drop=True, inplace=True)
    return sold_data

In [231]:
%%time

all_sold_houses = False

for branch in branches:
    print(branch)
    _branch = branch.replace(" ", "-")
    if all_sold_houses is False:
        all_sold_houses = get_rightmove_sold_properties_by_location(_branch)
        
    else:
        houses_sold_in_location = get_rightmove_sold_properties_by_location(_branch)
        all_sold_houses = pd.concat(
            [houses_sold_in_location, all_sold_houses], axis=0
        )
        
all_sold_houses.reset_index(drop=True, inplace=True)
all_sold_houses

Abbey Wood
Abington
Andover
Ashford
Baildon
Barnsley
Basingstoke
Beccles
Bedford
Bedworth
Beeston
Belper
Bexleyheath
Blaby
Blackburn
Blackheath
Bognor Regis
Bothwell
Brighton and Hove
Broadstairs
Bromley
Bromsgrove
Burton on Trent
Bury
Camborne
Canterbury
Carlisle
Castle Bromwich
Chadwell Heath
Chapeltown
Chatham
Chester-le-Street
Coalville
Coatbridge
Consett
Cornwall New Homes
Crawcrook


C:\Users\bradj\AppData\Local\Temp\ipykernel_29968\1077198389.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(page.content, "html.parser")


Crewe
Dalgety Bay
Dalkeith
Darlington
Dartford
Darwen
Deal
Denton
Doncaster
Dudley
Dumfries
Dundee
Dunfermline
Dunstable
Durham
Duston
Eastbourne
Edinburgh
Egham
Elgin
Eltham
Exmouth
Falkirk
Faversham
Forest Gate
Gillingham
Glenrothes
Goole
Gorleston
Gravesend
Hamilton
Hastings
Hemel Hempstead
Hinckley
Hitchin
Hoo
Hornchurch
Houghton-le-Spring
Ilkeston
Inverness
Keighley
Kettering
Keynsham
Kingsthorpe
Kirkby-in-Ashfield
Kirkcaldy
Lanark
Langley
Leeds
Leicester
Leven
Lincoln
Livingston
Longfield
Longton
Loughborough
Low Fell
Lowestoft
Maidstone
Margate
Midsomer Norton
Montrose
New Cross
Newton Abbot
North Hykeham
Northampton
Northampton Student Living
Northfield


C:\Users\bradj\AppData\Local\Temp\ipykernel_29968\1077198389.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(page.content, "html.parser")


Northumberland Heath
Norwich
Nottingham
Nuneaton
Orpington
Oswestry
Perton
Plymouth
Polegate
Radcliffe
Rainham
Ramsgate
Rochester
Romford
Rowlands Gill
Shaw
Sheerness
Sidmouth
Sittingbourne
Skelmersdale
Southampton
Southsea
St Austell
Stafford
Stechford
Stirling
Sunderland
Surbiton
Sutton
Sutton Lettings
Sutton-on-Sea


C:\Users\bradj\AppData\Local\Temp\ipykernel_29968\1077198389.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(page.content, "html.parser")


Swadlincote
Swanley
Swinton
Tunbridge Wells
Tunstall
Walderslade
Walkden
Wallington
Wallsend
Walthamstow
Washington
Waterlooville
Watford
Wednesfield
Wellington
West Denton
Whickham
Whitehaven
Whitley Bay
Whitton
Wigton
Winlaton
Wishaw
Wollaton
Wolverhampton
Wombwell
York
CPU times: total: 7.66 s
Wall time: 55.8 s


,location,address,property type,bedrooms,sale price,sale date
0,York,"29, Skiddaw, York YO24 2SZ",Semi-Detached,2,"£237,500",15 Feb 2023
1,York,"29, Skiddaw, York YO24 2SZ",Semi-Detached,2,"£168,500",20 Aug 2015
2,York,"29, Skiddaw, York YO24 2SZ",Semi-Detached,2,"£154,000",15 Aug 2014
3,York,"Apartment 12, Hunter House, 57, Goodramgate, Y...",Flat,null,"£297,000",15 Feb 2023
4,York,"Apartment 12, Hunter House, 57, Goodramgate, Y...",Flat,null,"£275,000",6 Feb 2020
...,...,...,...,...,...,...
9960,Abbey-Wood,"1, Nassau Path, London, Greater London SE28 8AN",Terraced,null,"£164,000",4 Jul 2005
9961,Abbey-Wood,"1, Nassau Path, London, Greater London SE28 8AN",Terraced,null,"£120,000",28 Oct 2002
9962,Abbey-Wood,"1, Nassau Path, London, Greater London SE28 8AN",Terraced,null,"£44,000",12 Jan 1996
9963,Abbey-Wood,"442, Bedonwell Road, London, Greater London SE...",Detached,3,"£615,000",25 Oct 2022


In [232]:
set(all_sold_houses["property type"])

{0, 'Detached', 'Flat', 'Semi-Detached', 'Terraced', 'Unknown'}

In [234]:
all_sold_houses.loc[all_sold_houses["property type"] == 0]

,location,address,property type,bedrooms,sale price,sale date
1747,0,0,0,0,0,0
3604,0,0,0,0,0,0
7816,0,0,0,0,0,0


In [235]:
all_sold_houses.loc[all_sold_houses["property type"] == "Unknown"]

,location,address,property type,bedrooms,sale price,sale date
244,Wishaw,"63, Cowie Place, Wishaw, Lanarkshire ML2 7US",Unknown,null,"£82,000",27 Feb 2023
245,Wishaw,"63, Cowie Place, Wishaw, Lanarkshire ML2 7US",Unknown,null,"£70,000",28 Aug 2019
246,Wishaw,"63, Cowie Place, Wishaw, Lanarkshire ML2 7US",Unknown,null,"£90,000",20 Oct 2008
247,Wishaw,"55, Cowie Place, Wishaw, Lanarkshire ML2 7US",Unknown,2,"£98,000",22 Feb 2023
248,Wishaw,"55, Cowie Place, Wishaw, Lanarkshire ML2 7US",Unknown,2,"£87,000",7 Nov 2018
...,...,...,...,...,...,...
8884,Bothwell,"5, Langside Road, Bothwell, Glasgow, Lanark G7...",Unknown,1,"£35,000",9 Apr 2003
8885,Bothwell,"5, Langside Road, Bothwell, Glasgow, Lanark G7...",Unknown,1,"£43,500",24 Jan 2003
8886,Bothwell,"28 Flat 33, Fairfield Lodge, Green Street, Bot...",Unknown,null,"£95,000",4 Jul 2022
8887,Bothwell,"28 Flat 33, Fairfield Lodge, Green Street, Bot...",Unknown,null,"£65,000",5 Sep 2018


In [236]:
all_sold_houses.loc[all_sold_houses["bedrooms"] == "null"]

,location,address,property type,bedrooms,sale price,sale date
3,York,"Apartment 12, Hunter House, 57, Goodramgate, Y...",Flat,null,"£297,000",15 Feb 2023
4,York,"Apartment 12, Hunter House, 57, Goodramgate, Y...",Flat,null,"£275,000",6 Feb 2020
5,York,"Apartment 12, Hunter House, 57, Goodramgate, Y...",Flat,null,"£251,867",24 Mar 2016
6,York,"8, Monkgate Cloisters, York YO31 7HY",Flat,null,"£268,125",10 Feb 2023
11,York,"18, Kingsway West, York YO24 4QA",Semi-Detached,null,"£210,000",3 Feb 2023
...,...,...,...,...,...,...
9946,Abbey-Wood,"Flat 3, 8, Hermitage Close, London, Greater Lo...",Flat,null,"£144,495",21 Nov 2003
9959,Abbey-Wood,"1, Nassau Path, London, Greater London SE28 8AN",Terraced,null,"£320,000",26 Oct 2022
9960,Abbey-Wood,"1, Nassau Path, London, Greater London SE28 8AN",Terraced,null,"£164,000",4 Jul 2005
9961,Abbey-Wood,"1, Nassau Path, London, Greater London SE28 8AN",Terraced,null,"£120,000",28 Oct 2002


In [245]:
all_sold_houses["address"][1000].split(", ")[-1]

'Hampshire PO8 8PU'